Hi Everyone
This will be the notebook for the Capstone Project in my IBM Data Science Professional Certification through Coursera

In [65]:
import pandas as pd
import numpy as np
print("Hello Capstone Project Course")

Hello Capstone Project Course


# This notebook covers the clustering and segmentation of Toronto neighborhoods

## This scrapes the information from the web page

In [66]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
# makes a request to the web page and gets its HTML
r = requests.get(url)
# stores the HTML page in 'soup', a BeautifulSoup object
soup = BeautifulSoup(r.content)

In [67]:
print(soup.prettify())

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en">
 <head>
  <meta charset="utf-8"/>
  <title>
   List of postal codes of Canada: M - Wikipedia
  </title>
  <script>
   document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"Xsl6YApAICkAAD-@iDAAAADA","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":958430791,"wgRevisionId":958430791,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Communications in Ontario","Postal codes in Canada","Toronto","Ontario

In [68]:
a = []
df = pd.DataFrame(columns = ['Postal Code', 'Borough', 'Neighborhood'])
for link in soup.find_all('td'):
    a.append(link.get_text())
    if len(a) == 3:
        df_length = len(df)
        df.loc[df_length] = a
        a = []

In [69]:
df.head()

,Postal Code,Borough,Neighborhood
0,M1A\n,Not assigned\n,\n
1,M2A\n,Not assigned\n,\n
2,M3A\n,North York\n,Parkwoods\n
3,M4A\n,North York\n,Victoria Village\n
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront\n"


In [70]:
df['Postal Code'] = df['Postal Code'].str.replace('\n', '')
df['Borough'] = df['Borough'].str.replace('\n', '')
df['Neighborhood'] = df['Neighborhood'].str.replace('\n', '')
df.head(3)

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,
1,M2A,Not assigned,
2,M3A,North York,Parkwoods


In [71]:
dfb=df[df.Borough != 'Not assigned']
dfb.head(15)

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"
11,M3B,North York,Don Mills
12,M4B,East York,"Parkview Hill, Woodbine Gardens"
13,M5B,Downtown Toronto,"Garden District, Ryerson"


In [72]:
dfb.shape

(114, 3)

In [73]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!pip install folium==0.5.0
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [86]:
filename = "http://cocl.us/Geospatial_data"

In [89]:
df_hoods=pd.read_csv(filename)
df_hoods.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [91]:
dfc = dfb.merge(df_hoods[['Postal Code','Latitude', 'Longitude']], on=['Postal Code'])
dfc.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [92]:
CLIENT_ID = 'B0DS25U1DJ0XKSWPYO3KBXGAVHYIIDKOU11VQTHLW5SODOVH' # your Foursquare ID
CLIENT_SECRET = 'TS2RZJURENOTLGQVST4ICDLTXPHOOJA3X0KVJNX344WBJ1LB' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: B0DS25U1DJ0XKSWPYO3KBXGAVHYIIDKOU11VQTHLW5SODOVH
CLIENT_SECRET:TS2RZJURENOTLGQVST4ICDLTXPHOOJA3X0KVJNX344WBJ1LB


In [12]:
address = 'Toronto, CAN'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.22.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


43.8184944 -79.3309468


In [19]:
search_query = 'All'
radius = 5000
print(search_query + ' OK. That is done')

All OK. That is done


In [20]:
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=B0DS25U1DJ0XKSWPYO3KBXGAVHYIIDKOU11VQTHLW5SODOVH&client_secret=TS2RZJURENOTLGQVST4ICDLTXPHOOJA3X0KVJNX344WBJ1LB&ll=43.8184944,-79.3309468&v=20180604&query=All&radius=5000&limit=30'

In [21]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5ed14a40f7706a001b9a7f6f'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Markham',
  'headerFullLocation': 'Markham',
  'headerLocationGranularity': 'city',
  'query': 'all',
  'totalResults': 116,
  'suggestedBounds': {'ne': {'lat': 43.86349444500004,
    'lng': -79.26869630032881},
   'sw': {'lat': 43.773494354999954, 'lng': -79.3931972996712}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4b833f4af964a5200fff30e3',
       'name': 'Hilton Toronto/Markham Suites Conference Centre & Spa',
       'location': {'address': '8500 Warden Avenue',
        'crossStreet': 'at Highway 7 E',
        'lat': 43.85431256098079,
        'lng': -79.335212

In [26]:
# assigning relevant part of JSON to VEGETARIAN venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

KeyError: 'venues'

In [25]:
dataframe.shape

(1, 12)